In [42]:
import pandas as pd
import pgeocode
from config import gkey
import requests
import json
import pymongo
import numpy as np

In [43]:
zips_df=pd.read_csv('sf_zips.csv')

In [44]:
zips_df["Latitude"] = ""
zips_df["Longitude"] = ""

In [45]:
tool = pgeocode.Nominatim('US')

In [46]:
tool_table=tool.query_postal_code(95822)

In [47]:
tool_table["latitude"]

38.5091

In [48]:
for index, row in zips_df.iterrows():
    zipcode = row['ZIPCode']
    tool_table=tool.query_postal_code(zipcode)
    lat=tool_table["latitude"]
    long=tool_table["longitude"]
    zips_df.loc[index, 'Latitude']=lat
    zips_df.loc[index, 'Longitude']=long
    

In [49]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.locations_mdb

In [50]:
api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in zips_df.iterrows():
    target_coordinates = f"{zips_df.loc[index, 'Latitude']},{zips_df.loc[index, 'Longitude']}" 
    target_radius = 50000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey}

# run a request using our params dictionary
    response = requests.get(api_url, params=params).json()
    
    
    db.locations.insert_one(
        {
            'zip_code': row['ZIPCode'],
            'type': row['Type'],
            'county': row['County'],
            'population': row['Population'],
            'area_codes': row['Area Code(s)'],
            'latitude': row['Latitude'],
            'longitude': row['Longitude'],
            'hotels': response["results"]
        }
    )

In [51]:
#print(db.locations.find({"rating":3})
#print(db.locations.find_one())
for x in db.locations.find({"zip_code":94102}):
    print(1)

1
1


In [52]:
#import json
#from pymongo import MongoClient
#collection_locations = db['locations']

#with open('currencies.json') as f:
#    file_data = json.load(f)

#locations.insert_one(file_data)

with open('data.txt', 'w') as outfile:
    json.dump(response["results"], outfile)

In [53]:
#for location in db.locations.find({"hotels.rating":4.1}):
#    for hotel in location["hotels"]:
#        if hotel["rating"] == 4.1:
#            print(json.dumps(hotel, sort_keys=True, indent=2))
            

In [54]:
with open('data.txt', 'w') as outfile:
    json.dump(response["results"], outfile)

In [58]:
collection_boba = db['boba']
with open ("Heesung_resources/boba_pika.json") as f:
    file_data = json.load(f)
    for record in file_data:
        collection_boba.insert_one(record)

In [61]:
df = pd.read_csv (r'Cait_resources/Resources/SF_Park_Scores.csv')
df.to_json ('sf_parks.json')

In [ ]:
collection_parks = db['parks']
with open ("Cait_resources/boba_pika.json") as f:
    file_data = json.load(f)
    for record in file_data:
        collection_boba.insert_one(record)

In [56]:
collection_restaurants = db['restaurants']

In [57]:
with open('currencies.json') as f:
    file_data = json.load(f)

# if pymongo < 3.0, use insert()
collection_currency.insert(file_data)
# if pymongo >= 3.0 use insert_one() for inserting one document
collection_currency.insert_one(file_data)
# if pymongo >= 3.0 use insert_many() for inserting many documents
collection_currency.insert_many(file_data)

client.close(

SyntaxError: unexpected EOF while parsing (<ipython-input-57-c04eba66293d>, line 11)